In [ ]:
from pathlib import Path
from datetime import datetime, timedelta

import polars as pl

import stock
from stock.trend_template import check_fundamental_trend_templates, check_technical_trend_templates

In [ ]:
def update_watch_list(date: datetime):
    csv_dir = stock.PROJECT_ROOT / "data" / "daily"
    watch_list = []
    for csv_path in sorted(csv_dir.glob("*.csv")):
        code = csv_path.stem
        if stock.trend_template.check_technical_trend_templates(
            code, cur_day=date,
        ) and stock.trend_template.check_fundamental_trend_templates(code, current_date=date):
            watch_list.append(code)
    return watch_list

In [ ]:
def calc_watch_list_duration_of(code: str, start_date: datetime, end_date: datetime = datetime.today()):
    duration = []
    date = start_date
    start = None
    while date <= end_date:
        if check_technical_trend_templates(code, cur_day=date) and check_fundamental_trend_templates(code, current_date=date):
            if start is None:
                start = date
        else:
            if start is not None:
                duration.append([start, date - timedelta(1)])
            start = None
        date += timedelta(days=1)

    if start is not None:
        duration.append([start, end_date])

    return duration


In [ ]:
date = datetime.today() - timedelta(days=365)
watch_list = update_watch_list(date)

In [ ]:
durations = []
nikkei_df = stock.kabutan.read_data_csv(stock.PROJECT_ROOT / "data" / "daily" / f"0000.csv", exclude_none=False)

In [ ]:
mean = 0.0
for idx, code in enumerate(watch_list):
    start_date = date - timedelta(days=365)
    # duration = calc_watch_list_duration_of(code, start_date=start_date)
    duration = durations[idx]

    data_csv = stock.PROJECT_ROOT / "data" / "daily" / f"{code}.csv"
    df = stock.kabutan.read_data_csv(data_csv, exclude_none=True)
    between_df = df.filter(pl.col("date").is_between(duration[0][0], duration[-1][1], closed="right"))
    # 
    start = between_df["high"][0]
    high_idx = between_df["high"].arg_max()
    high = between_df["high"][high_idx]
    high_date = between_df["date"][high_idx]
    #
    nikkei_between_df = nikkei_df.filter(pl.col("date").is_between(duration[0][0], duration[-1][1], closed="right"))
    nikkei_start = nikkei_between_df["high"][0]
    nikkei_high = nikkei_between_df["high"].max()
    
    print(code, high / start, nikkei_high / nikkei_start, duration[0][0].strftime("%Y/%m/%d"), duration[-1][1].strftime("%Y/%m/%d"))
    #diff = (high / start) - (nikkei_high / nikkei_start)
    mean += high / start
    #durations.append(duration)

In [ ]:
mean / len(watch_list)

In [ ]:
durations

In [ ]:
start, high

In [ ]:
watch_list

In [ ]:
prev_date = date - timedelta(days=1)
prev_watch_list = update_watch_list(prev_date)